### Ingest pit_stops.json file

##### Step 1 - Read the Json file using the spark dataframe reader API - Multiline Json

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col, lit
from pyspark.sql.functions import to_timestamp, concat, col, current_timestamp, to_date

In [0]:
lap_times_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                      ])


In [0]:
lap_times_df = spark.read \
.schema(lap_times_schema) \
.csv(f"{raw_folder_path}/{v_file_date}/lap_times")

In [0]:
# display(lap_times_df)

####Renaming columns

In [0]:
laptime_ingetion_date_df = add_ingestion_date(lap_times_df)

In [0]:
final_df = laptime_ingetion_date_df.withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("driverId", "driver_id") \
.withColumn("data_source", lit(v_data_source)) \
.withColumn("file_date", lit(v_file_date))

In [0]:
overwrite_partition(final_df, "f1_processed", "lap_times", "race_id")

In [0]:
# final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/lap_times")

In [0]:
# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.lap_times")

In [0]:
# display(spark.read.parquet(f"{processed_folder_path}/lap_times"))

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
SELECT race_id, COUNT(1)
FROM f1_processed.lap_times
GROUP BY race_id
ORDER BY race_id DESC